In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1358 entries, 0 to 1357
Data columns (total 16 columns):
grade_A_Component_1    1358 non-null int64
grade_A_Component_2    1358 non-null int64
max_luminosity         1358 non-null int64
thickness              1358 non-null int64
xmin                   1358 non-null int64
xmax                   1358 non-null int64
ymin                   1358 non-null int64
ymax                   1358 non-null int64
pixel_area             1358 non-null int64
log_area               1358 non-null float64
x_component_1          1358 non-null int64
x_component_2          1358 non-null int64
x_component_3          1358 non-null int64
x_component_4          1358 non-null int64
x_component_5          1358 non-null int64
class                  1358 non-null int64
dtypes: float64(1), int64(15)
memory usage: 169.8 KB


In [4]:
from sklearn.model_selection import train_test_split

In [5]:
train2 = train.iloc[:1000,:]
val = train.iloc[1000:,:]

In [6]:
train2.shape

(1000, 16)

In [7]:
val.shape

(358, 16)

In [8]:
# from featexp import get_univariate_plots

# get_univariate_plots(data=train,target_col='class',data_test=val,bins=10)

In [9]:
# from featexp import get_trend_stats

# stats = get_trend_stats(data=train,target_col='class',data_test=val)

In [10]:
# stats

In [11]:
train.columns

Index(['grade_A_Component_1', 'grade_A_Component_2', 'max_luminosity',
       'thickness', 'xmin', 'xmax', 'ymin', 'ymax', 'pixel_area', 'log_area',
       'x_component_1', 'x_component_2', 'x_component_3', 'x_component_4',
       'x_component_5', 'class'],
      dtype='object')

In [12]:
features_list = ['max_luminosity','thickness', 'xmin', 'xmax','ymin', 'ymax', 'pixel_area', 'log_area','x_component_1', 'x_component_2', 'x_component_3', 'x_component_4','x_component_5']

In [13]:
X = train[features_list]
y = train['class']
test = test[features_list]

In [14]:
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2,random_state=101)

In [15]:
from sklearn.preprocessing import StandardScaler

In [16]:
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_val = sc.fit_transform(X_val)

In [17]:
from xgboost import XGBClassifier

/home/yash/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/yash/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [18]:
xgbc = XGBClassifier(learning_rate=0.1,n_estimators=100)

In [19]:
from sklearn.metrics import log_loss

In [20]:
xgbc.fit(X_train,y_train)
xgbc_pred = xgbc.predict_proba(X_val)
xgbc_score = log_loss(y_val,xgbc_pred)

In [21]:
print(xgbc_score)

0.3149532938721891


In [22]:
for feat_imp,param in zip(xgbc.feature_importances_,features_list):
    print(feat_imp,param)

0.034643937 max_luminosity
0.04134077 thickness
0.034019727 xmin
0.030967478 xmax
0.0249384 ymin
0.021259068 ymax
0.02315349 pixel_area
0.025714822 log_area
0.16891061 x_component_1
0.16693996 x_component_2
0.2335242 x_component_3
0.10340897 x_component_4
0.09117854 x_component_5


In [23]:
xgbc.fit(X,y)
predis = xgbc.predict_proba(test)

In [24]:
predics = pd.DataFrame(predis,columns=['1','2'])

filename = 'submit_fxp_3.xlsx'
predics.to_excel(filename,index=False)

####Inference = Better results on same model with dropping feats - grade_comp_a_1/2